## Compute source time course (STC) from Raw and Epochs data object
#### Input: *_preprocessed-raw.fif, -epo.fif
####  Output: *.stc

In [1]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import preprocess, utils, source_localization
%matplotlib inline

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


In [2]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in processed raw data
data_dir = Path("../../Data")
data_path = data_dir / 'Processed Data/'
epo_path = data_dir / 'Data/Processed Data/' / time_win_path
EC_resting_save_path = data_dir / "STC_EC_resting"
EO_resting_save_path = data_dir / "STC_EO_resting"
zscored_epochs_save_path = data_dir / "STC_zscored_Epochs" / time_win_path
save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 

# Create save paths
[os.makedirs(path) for path in save_paths if not os.path.exists(path)]

[-2.5,0.0,2.5]


[]

In [3]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_path,os.pardir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [4]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 36
Controls: 25
Total: 61


In [5]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

In [6]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh'] # Right Medial-OFC

In [7]:
for sub_id in sub_ids:
    EC_bool = False if sub_id in no_eyes_open else True
    EO_bool = False if sub_id in no_eyes_closed else True
    label_ts_All_Conds, nan_subjects = source_localization.to_source(sub_id,data_path,
                                      zscored_epochs_save_path,
                                      EC_resting_save_path,
                                      EO_resting_save_path,
                                      roi_names,
                                      times_tup,
                                      return_zepochs=True,
                                      return_EC_resting= False, #EC_bool,
                                      return_EO_resting= False, #EO_bool,
                                      average_dipoles=True,
                                      )

Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 64 channels.
    64 out of 64 channels remain after picking
Selected 64 channels
Creating the depth weighting matrix...
    64 EEG channels
    limit = 20485/20484 = 2.207463
    scale = 125765 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 5.6e-13 (2.2e-16 eps * 64 dim * 40  max singular value)
    Estimated rank (eeg): 60
    EEG: rank 60 computed from 64 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of w

In [ ]:
label_ts_All_Conds

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])